### Evaluation
이 예제에서는 Evaluation을 해보겠습니다. `rouge`같은 벤치마크가 아닌 LLM을 이용한 평가 방법입니다.  
물론 벤치마크 평가가 사장된 것은 아닙니다만, 현재는 LLM을 통한 평가 방법을 일반적으로 많이 사용하고 있습니다.  

이 예제는 Judging LLM-as-a-Judge 논문 ([https://arxiv.org/pdf/2306.05685](https://arxiv.org/pdf/2306.05685))과 [Openai evals](https://github.com/openai/evals)를 바탕으로 작성되었습니다.  
프롬프트 또한 논문을 기반으로 작성되었습니다.  

평가는 아래의 순서로 진행됩니다.  
1. 튜닝전 모델과 튜닝 후 모델을 불러옵니다.
2. 작성해놓은 평가 프롬프트를 input으로 하여 각각의 모델로 output을 생성합니다.
3. ChatGPT-4o 각 모델들의 output을 전달하여 평가하도록 요청합니다.

In [12]:
!pip install --quiet\
peft\
accelerate\
flash-attn\
transformers\
openai\
python-dotenv\
selenium\
colorama

In [13]:
import os
import sys
import json
import copy
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import AutoPeftModelForCausalLM
import torch
from google.colab import userdata, drive
# utils, prompts 커스텀 모듈 사용을 위해 구글 드라이브에 마운트합니다.
drive.mount('/content/drive')

# git clone 받은 위치로 지정합니다.
path = "/content/drive/MyDrive/instruction-tuning-with-rag-example"

sys.path.append(path)
import utils
import prompts

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### 1. 튜닝전 모델과 튜닝 후 모델을 불러옵니다

In [14]:
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it")
base_model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2b-it",
    device_map="cuda",
    torch_dtype=torch.bfloat16,
    token=userdata.get("HF_TOKEN"),
    attn_implementation="flash_attention_2"
)

finetuned_model = AutoPeftModelForCausalLM.from_pretrained(
    "aiqwe/gemma-2b-it-example-v1",
    device_map="cuda",
    torch_dtype=torch.bfloat16,
    attn_implementation="flash_attention_2"
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

### 2. 작성해놓은 평가 프롬프트를 input으로 하여 각각의 모델로 output을 생성합니다.

In [15]:
# 평가 데이터 불러오기
with open(os.path.join(path, "data/eval_dataset.txt"), "r") as f:
    eval_inputs = f.readlines()

eval_inputs = [inputs.strip("\n") for inputs in eval_inputs]
eval_inputs

['정비 사업의 진행 절차를 알려줘.',
 '실거주 의무가 뭔가요?',
 '신속통합기획 진행중인 곳이 어디인가요?',
 '청약에 당첨되려면 뭘 해야할까요?',
 '부동산 주요 정책은 뭐가 있나요?',
 '전세 사기에 대비하는 방법을 알려주세요.',
 '부동산 매도시 양도세에 대해 설명해주세요.',
 '전매 제한이 뭔가요?',
 '주택 담보 대출을 받으려면 어떻게 해야할까요?',
 'DSR, DTI, LTV에 대해 설명해주세요.',
 '디에이치 방배는 언제 분양되나요?',
 '한남 뉴타운 진행사항이 어떻게 될까요?',
 '분양가 상한제시 어떤 규제를 받나요?',
 '임차인 우서 변제권이 뭔가요?',
 '전세 중개수수료는 얼마나 내야하나요?']

In [16]:
# Base모델 / Fine-tuning 모델 모두 Chat Template을 사용했기 때문에 Inference때도 Chat Template을 사용해줍니다.
inputs = [tokenizer.apply_chat_template(
    conversation=[
        {"role": "user", "content": query}
    ],
    add_generation_prompt=True,
    return_tensors="pt",
    tokenize=False
) for query in eval_inputs]

In [17]:
inputs

['<bos><start_of_turn>user\n정비 사업의 진행 절차를 알려줘.<end_of_turn>\n<start_of_turn>model\n',
 '<bos><start_of_turn>user\n실거주 의무가 뭔가요?<end_of_turn>\n<start_of_turn>model\n',
 '<bos><start_of_turn>user\n신속통합기획 진행중인 곳이 어디인가요?<end_of_turn>\n<start_of_turn>model\n',
 '<bos><start_of_turn>user\n청약에 당첨되려면 뭘 해야할까요?<end_of_turn>\n<start_of_turn>model\n',
 '<bos><start_of_turn>user\n부동산 주요 정책은 뭐가 있나요?<end_of_turn>\n<start_of_turn>model\n',
 '<bos><start_of_turn>user\n전세 사기에 대비하는 방법을 알려주세요.<end_of_turn>\n<start_of_turn>model\n',
 '<bos><start_of_turn>user\n부동산 매도시 양도세에 대해 설명해주세요.<end_of_turn>\n<start_of_turn>model\n',
 '<bos><start_of_turn>user\n전매 제한이 뭔가요?<end_of_turn>\n<start_of_turn>model\n',
 '<bos><start_of_turn>user\n주택 담보 대출을 받으려면 어떻게 해야할까요?<end_of_turn>\n<start_of_turn>model\n',
 '<bos><start_of_turn>user\nDSR, DTI, LTV에 대해 설명해주세요.<end_of_turn>\n<start_of_turn>model\n',
 '<bos><start_of_turn>user\n디에이치 방배는 언제 분양되나요?<end_of_turn>\n<start_of_turn>model\n',
 '<bos><start_of_turn>user\n한남 뉴타운 진행사항이 

In [18]:
from utils import generate

# Base 모델 / 튜닝한 모델의 output generate하기
input_text = tokenizer(inputs, return_tensors="pt", padding=True).to(base_model.device)

# base modeld에서 배치로 generate
outputs = base_model.generate(**input_text, max_new_tokens=512, repetition_penalty = 1.5, temperature=0)
decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)
# output만 가져오기
base_decoded = [text.split("model")[1] for text in decoded]

# fine-tuning 모델 배치로 generate
outputs = finetuned_model.generate(**input_text, max_new_tokens=512, repetition_penalty = 1.5, temperature=0)
decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)
finetuned_decoded = [text.split("model")[1] for text in decoded]

# 저장하기
eval_dataset = dict(inputs=eval_inputs, completion1=base_decoded, completion2=finetuned_decoded)
utils.jsave(data=eval_dataset, save_path=os.path.join(path, "data/eval_dataset.json"), mode="w", indent=4)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


In [19]:
eval_dataset = utils.jload(os.path.join(path, "data/eval_dataset.json"))

In [20]:
from colorama import Fore, Style

idx = 4

print(Fore.MAGENTA + "INPUT:" + Style.RESET_ALL)
print(eval_dataset['inputs'][idx] + "\n")

print(Fore.MAGENTA + "BASE MODEL:" + Style.RESET_ALL)
print(eval_dataset['completion1'][idx] + "\n")

print(Fore.MAGENTA + "FINETUNED MODEL:" + Style.RESET_ALL)
print(eval_dataset['completion2'][idx])

INPUT:
부동산 주요 정책은 뭐가 있나요?

BASE MODEL:

**부동산 주요 정책**

* **국민주택 공급 지원:** 국민이 부동산에 거주하기 위해 필요한 다양한 수준의 저비용과 장기적인 보증을 제공합니다.
* **부동산 용도 지정:** 특정 지역에서 특정 용도로 사용할 수 있는지 명확하게 설정하여 개발 및 활용을 유도합니다.
* **부동산 관리 시스템 개선:** 효율성과 안전성을 높이기 위한 철저하고 투명한 부동산 관리 시스템 구축.
* **거래 통제:** 부동산 가격의 과도한 상승을 방지하기 위하여 거래를 제어하는 기반 마련.
* **자연스러운 개발 환경 조성:** 자원 활용을 고려한 부동산 계획, 교통, 교육 등의 시스템적 지원을 강화합니다.
* **부동산 투자 확대:** 국내외 투자자에게 부동산 투자가 더욱 안전하고 효과적으로 이루어질 수 있도록 지원을 제공합니다.
* **부동산 법률 완화:** 부동산 관련 법률을 간소화하고 효과적으로 운영하도록 도와줍니다.

FINETUNED MODEL:

현재 부동산 관련 중요한 정책으로는 다주택자에 대한 취득세 중과 완화, 임대사업 활성화 방안 도출 등이 있습니다. 특히 청년 및 신혼부부를 위한 저금리 대출 지원도 포함되어 있으며, 이러한 제도들은 실수요자들에게 큰 도움을 제공하고 있습니다. 또한, 재건축·재개발의 원활한 진행을 위해 '공공기여율' 개선 조치와 같은 법적 요소들이 마련되고 있습니다. 따라서 현재 가장 필요로 하는 것은 해당 지역에서 시설물 보존 관리 기법 강화입니다. 예를 들어, 공원이나 산림에는 유지보수 계획 수립 과정부터 현장 점검 확대까지 철저하게 준비됩니다. 전반적으로 국민들의 생활 편의를 높이고 환경 을 보호하는 데 초점을 맞춘 것입니다.



Judging LLM-as-a-Judge([https://arxiv.org/pdf/2306.05685](https://arxiv.org/pdf/2306.05685))논문에 의하면 2개의 Assistant를 비교할 때 순서가 영향을 줄 수 있다고 합니다.  
따라서 base model과 finetuned model의 순서를 바꿔서 한번 더 평가를 받습니다.  
+ 첫 번째 평가 : (A: Base Model, B: Finetuned Model)  
+ 두 번째 평가 : (A: FineTuned Model, B: Finetuned Model)  

따라서 FineTuning한 모델이 평가가 좋으려면, 첫번째 평가는 B가 많아야하고 두번째는 A가 많아야합니다.  

In [21]:
from colorama import Fore, Style

# 프롬프트 샘플
for input, out1, out2 in zip(eval_dataset['inputs'], eval_dataset['completion1'], eval_dataset['completion2']):
    print(Fore.MAGENTA + "첫번째 평가할 프롬프트" + Style.RESET_ALL)
    print(prompts.EVAL_BATTLE_PROMPT.format(question=input, answer_a=out1, answer_b=out2))
    print("\n\n")
    print(Fore.MAGENTA + "두번째 평가할 프롬프트" + Style.RESET_ALL)
    print(prompts.EVAL_BATTLE_PROMPT.format(question=input, answer_a=out2, answer_b=out1))
    break

첫번째 평가할 프롬프트

[System]
공정한 심사위원이 되어 2개의 AI 모델이 제공하는 답변의 품질을 평가하세요.
사용자의 지시를 잘 따르고, 사용자의 질문에 더 잘 답변하는 Assistant를 선택하세요.
답변의 유용성, 관련성, 정확성, 깊이, 창의성 및 세부 수준과 같은 요소를 고려하여 평가해야 합니다.
가장 중요하게 고려할 사항은 정확성입니다.
최대한 객관적으로 판단해야 하며, 어떠한 입장의 bias도 있어선 안됩니다.
답변이 제시된 순서가 판단에 영향을 미치지 않도록 해야합니다.
답변의 길이가 평가에 영향을 미치지 않도록 합니다.
특정 Assistant의 이름을 선호해서는 안됩니다.
가능한 객관적으로 평가해야합니다.

평가 후 최종 평점을 산출해야 합니다:
- A Assistant의 답변이 더 좋으면 "A", B Assistant의 답변이 더 좋으면 "B", 동점이면 "C"로 표기합니다.

평가후 JSON 포맷에따라 평점과 설명을 출력하세요.
Markdown 양식은 제거하고 출력하세요.
출력할 JSON 스키마는 아래와 같습니다.
{"selected_assistant": selected_assistant: str, "reason": reason: str}
- selected_assistant : 선택한 Assistant
- reason : selected_assistant를 선택한 이유

[사용자 질문]
정비 사업의 진행 절차를 알려줘.
[사용자 질문]
[Assistant A의 답변 시작]

**정비 사업 진행 절차**

1. **목표 설정:** 정비 사업을 수행하기 위한 목표와 기대 결과를 명확하게 설정해야 합니다.


2. **시스템 분석 및 평가:** 현재 시스템의 문제점과 기능 부족을 파악하고, 이에 대한 해결책을 도출해야 합니다.


3. **계획 마련:** 정비 계획서를 작성하여 정비 과정에서 필요한 모든 작업을 상세하게 기술해야 합니다.


4. **자원 확보:** 정비 비용을 예산으로 책정하고, 필요한 자원(인력, 장비 등)을 구축해

### 3. Base 모델과 Battle

In [9]:
openai_key = userdata.get("OPENAI_API_KEY")

scores_1 = []
scores_2 = []
for input, out1, out2 in zip(eval_dataset['inputs'], eval_dataset['completion1'], eval_dataset['completion2']):
    result = utils.get_completion(prompts.EVAL_BATTLE_PROMPT.format(question=input, answer_a=out1, answer_b=out2), api_key=openai_key, model="gpt-4o")
    scores_1.append(result)
    result = utils.get_completion(prompts.EVAL_BATTLE_PROMPT.format(question=input, answer_a=out2, answer_b=out1), api_key=openai_key, model="gpt-4o")
    scores_2.append(result)

In [13]:
# 이긴 Assistant 보기
winner1 = [json.loads(s1)['selected_assistant'] for s1 in scores_1]
winner2 = [json.loads(s2)['selected_assistant'] for s2 in scores_2]

In [22]:
import pandas as pd

winner_df = pd.DataFrame.from_dict({"eval1": winner1, "eval2": winner2})
winner_df

,eval1,eval2
0,A,B
1,B,A
2,B,A
3,B,A
4,B,B
5,B,A
6,B,A
7,A,A
8,A,B
9,B,A


In [40]:
# Finetuning 모델의 승리 갯수
wins = (winner_df['eval1'] == "B").sum() + (winner_df['eval2'] == "A").sum()
total = winner_df['eval1'].count() * 2
print(f"승률 : {wins / total:.2%}")


승률 : 73.33%


### 4. LLM에게 점수받기

In [10]:
openai_key = userdata.get("OPENAI_API_KEY")

scores = []
for input, out2 in zip(eval_dataset['inputs'], eval_dataset['completion2']):
    result = utils.get_completion(prompts.EVAL_SCORE_PROMPT.format(question=input, answer=out2), api_key=openai_key, model="gpt-4o")
    scores.append(result)

In [30]:
json.loads(scores[0])

{'score': 3,
 'reason': "답변은 정비 사업의 주요 단계를 다루고 있지만, 여러 부분에서 부정확하거나 불명확한 설명이 포함되어 있습니다. 예를 들어, '조합 설립' 단계에서 '이주민 모집'이라는 표현은 부적절하며, '소유권자 동호수 확보'라는 표현도 모호합니다. 또한, '인허가와 허가 사항 확인' 부분에서 '건축심원일 현재 시설물 보존등기 상태여야 합니다'라는 문장은 이해하기 어렵습니다. '분양 계약 체결' 부분도 구체적인 설명이 부족합니다. 전반적으로 정확성과 깊이 면에서 개선이 필요합니다."}

RLHF 학습이 되지 않았는데, 프롬프트상 RLHF를 평가하는 내용이 있어서 전반적으로 점수가 낮은 것 같습니다.

In [26]:
scores_list = [int(json.loads(s)['score']) for s in scores]

In [27]:
scores_list

[3, 2, 6, 3, 3, 3, 2, 2, 6, 2, 2, 3, 2, 1, 1]